# Azure AI Agents с поддержкой Model Context Protocol (MCP)

Этот ноутбук демонстрирует, как использовать Azure AI Agents с инструментами Model Context Protocol (MCP) для создания интеллектуального агента, который может использовать внешние MCP-серверы для расширенных возможностей.


## Установка необходимых пакетов NuGet

Сначала необходимо установить пакет Azure AI Agents Persistent, который предоставляет основную функциональность для работы с Azure AI Agents.


## Преимущества аутентификации без ключей

Этот ноутбук демонстрирует **аутентификацию без ключей**, которая предоставляет несколько преимуществ:
- ✅ **Нет необходимости управлять API-ключами** - Используется аутентификация на основе идентификации Azure
- ✅ **Повышенная безопасность** - Секреты не хранятся в коде или конфигурации
- ✅ **Автоматическая ротация учетных данных** - Azure управляет жизненным циклом учетных данных
- ✅ **Доступ на основе ролей** - Используется Azure RBAC для детализированного управления доступом

`DefaultAzureCredential` автоматически использует лучший доступный источник учетных данных:
1. Управляемая идентификация (при запуске в Azure)
2. Учетные данные Azure CLI (во время разработки)
3. Учетные данные Visual Studio
4. Переменные окружения (если настроены)


In [1]:
#r "nuget: Azure.AI.Agents.Persistent, 1.1.0-beta.4"

Installed Packages Azure.AI.Agents.Persistent, 1.1.0-beta.4

Установите пакет Azure Identity для аутентификации с сервисами Azure, используя DefaultAzureCredential.


In [2]:
#r "nuget: Azure.Identity, 1.14.2"

Installed Packages Azure.Identity, 1.14.2

## Импорт необходимых пространств имен

Импортируйте необходимые пространства имен для Azure AI Agents и Azure Identity.


In [3]:
using Azure.AI.Agents.Persistent;
using Azure.Identity;

## Настройка клиента Azure AI Agent (Аутентификация без ключа)

Настройте переменные конфигурации и создайте PersistentAgentsClient, используя **аутентификацию без ключа**:
- **projectEndpoint**: Конечная точка проекта Azure AI Foundry
- **modelDeploymentName**: Имя развернутой модели ИИ (GPT-4.1 nano)
- **mcpServerUrl**: URL сервера MCP (Microsoft Learn API)
- **mcpServerLabel**: Метка для идентификации сервера MCP
- **DefaultAzureCredential**: Использует управляемую идентификацию, Azure CLI или другие источники учетных данных (API-ключи не требуются)


In [ ]:
var projectEndpoint = "Your Azure AI Foundry Project Endpoint";
var modelDeploymentName = "Your Azure OpenAI Model Deployment Name";
var mcpServerUrl = "https://learn.microsoft.com/api/mcp";
var mcpServerLabel = "mslearn";
PersistentAgentsClient agentClient = new(projectEndpoint, new DefaultAzureCredential());

## Создание определения инструмента MCP

Создайте определение инструмента MCP, который подключается к серверу Microsoft Learn MCP. Это позволит агенту получать доступ к контенту и документации Microsoft Learn.


In [ ]:
MCPToolDefinition mcpTool = new(mcpServerLabel, mcpServerUrl);

## Создание AI-агента

Создайте постоянного AI-агента с указанной моделью и инструментами MCP. Агент настроен следующим образом:
- Модель GPT-4.1 nano
- Инструкции по использованию инструментов MCP для помощи
- Доступ к серверу Microsoft Learn MCP


In [ ]:
PersistentAgent agent = await agentClient.Administration.CreateAgentAsync(
   model: modelDeploymentName,
   name: "my-learn-agent",
   instructions: "You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
   tools: [mcpTool]
   );

## Создание темы и отправка сообщения

Создайте тему разговора и отправьте сообщение пользователю с вопросом о разнице между Azure OpenAI и OpenAI. Это проверит способность агента использовать инструменты MCP для предоставления точной информации.


In [7]:
PersistentAgentThread thread = await agentClient.Threads.CreateThreadAsync();

// Create message to thread
PersistentThreadMessage message = await agentClient.Messages.CreateMessageAsync(
    thread.Id,
    MessageRole.User,
    "What's difference between Azure OpenAI and OpenAI?");

## Настройка ресурсов инструмента MCP (без ключей)

Настройте ресурсы инструмента MCP. Для полностью безключевого подхода вы можете убрать пользовательские заголовки, если сервер MCP поддерживает аутентификацию на основе Azure Identity. Пример ниже показывает, как добавить заголовки при необходимости, но для безключевых сценариев они могут быть не нужны.


In [ ]:
// Option 1: Completely keyless (if MCP server supports Azure identity)
MCPToolResource mcpToolResource = new(mcpServerLabel);
ToolResources toolResources = mcpToolResource.ToToolResources();

// Option 2: With custom headers (if still needed for specific MCP servers)
// MCPToolResource mcpToolResource = new(mcpServerLabel);
// mcpToolResource.UpdateHeader("Authorization", "Bearer <your-token>");
// ToolResources toolResources = mcpToolResource.ToToolResources();

## Запуск агента

Создайте и запустите процесс для обработки сообщения пользователя. Агент будет использовать настроенные инструменты и ресурсы MCP для формирования ответа.


In [9]:
ThreadRun run = await agentClient.Runs.CreateRunAsync(thread, agent, toolResources);

## Мониторинг выполнения и обработка утверждений инструментов (без ключей)

Следите за статусом выполнения агента и обрабатывайте необходимые утверждения инструментов. Этот цикл:
1. Ожидает завершения выполнения или необходимости действий
2. Автоматически утверждает вызовы инструментов MCP, когда это требуется
3. Для аутентификации без ключей заголовки могут быть не нужны, если сервер MCP поддерживает идентификацию Azure


In [ ]:
while (run.Status == RunStatus.Queued || run.Status == RunStatus.InProgress || run.Status == RunStatus.RequiresAction)
{
    await Task.Delay(TimeSpan.FromMilliseconds(1000));
    run = await agentClient.Runs.GetRunAsync(thread.Id, run.Id);

    if (run.Status == RunStatus.RequiresAction && run.RequiredAction is SubmitToolApprovalAction toolApprovalAction)
    {
        var toolApprovals = new List<ToolApproval>();
        foreach (var toolCall in toolApprovalAction.SubmitToolApproval.ToolCalls)
        {
            if (toolCall is RequiredMcpToolCall mcpToolCall)
            {
                Console.WriteLine($"Approving MCP tool call: {mcpToolCall.Name}");
                
                // Option 1: Keyless approval (no headers needed)
                toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true));
                
                // Option 2: With headers (if required by specific MCP server)
                // toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true)
                // {
                //     Headers = { ["Authorization"] = "Bearer <your-token>" }
                // });
            }
        }

        if (toolApprovals.Count > 0)
        {
            run = await agentClient.Runs.SubmitToolOutputsToRunAsync(thread.Id, run.Id, toolApprovals: toolApprovals);
        }
    }
}

Approving MCP tool call: microsoft_docs_search


## Отображение результатов беседы

Получите и отобразите все сообщения в переписке, показывая как вопрос пользователя, так и ответ агента. Сообщения отображаются в хронологическом порядке с отметками времени и указанием роли.


In [12]:
using Azure;

AsyncPageable<PersistentThreadMessage> messages = agentClient.Messages.GetMessagesAsync(
    threadId: thread.Id,
    order: ListSortOrder.Ascending
);

await foreach (PersistentThreadMessage threadMessage in messages)
{
    Console.Write($"{threadMessage.CreatedAt:yyyy-MM-dd HH:mm:ss} - {threadMessage.Role,10}: ");
    foreach (MessageContent contentItem in threadMessage.ContentItems)
    {
        if (contentItem is MessageTextContent textItem)
        {
            Console.Write(textItem.Text);
        }
        else if (contentItem is MessageImageFileContent imageFileItem)
        {
            Console.Write($"<image from ID: {imageFileItem.FileId}>");
        }
        Console.WriteLine();
    }
}

2025-07-16 06:39:43 -       user: What's difference between Azure OpenAI and OpenAI?
2025-07-16 06:39:51 -  assistant: The main difference between Azure OpenAI and OpenAI lies in their deployment, management, and integration options:

1. **Azure OpenAI**:
   - A cloud service offered through Microsoft Azure.
   - Provides access to OpenAI models with additional enterprise features like security, compliance, and scale.
   - Allows integration with other Azure services, enabling seamless use within existing Azure-based solutions.
   - Offers managed deployment, monitoring, and support within the Azure ecosystem.
   - Suitable for organizations looking for enterprise-grade security, compliance, and regional availability.

2. **OpenAI (OpenAI API)**:
   - A standalone API service provided directly by OpenAI.
   - Accessible via the OpenAI platform without the need for Azure.
   - Focused on providing GPT models, DALL-E, etc., primarily for developers and researchers.
   - Suitable for indi


---

**Отказ от ответственности**:  
Этот документ был переведен с помощью сервиса автоматического перевода [Co-op Translator](https://github.com/Azure/co-op-translator). Несмотря на наши усилия по обеспечению точности, пожалуйста, учитывайте, что автоматические переводы могут содержать ошибки или неточности. Оригинальный документ на его родном языке следует считать авторитетным источником. Для получения критически важной информации рекомендуется профессиональный перевод человеком. Мы не несем ответственности за любые недоразумения или неправильные интерпретации, возникшие в результате использования данного перевода.
